# SRP
## single-responsibility principle

Каждый объект должен иметь одну ответственность и эта ответственность должна быть полностью инкапсулирована в класс. Все его поведения должны быть направлены исключительно на обеспечение этой ответственности.

## Пример

Мы пишем программу для аналитики данных.

В первой итерации, разработчики реализовали подключение к базе данных:

In [1]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column

In [2]:
# Упрощенная база данных
DATA = {
    'price': [100, 200, 300],
    'item': ['пиво', 'рыба', 'раки']
}

In [3]:
db = DB('0.0.0.0', DATA)
price = db.select_column('price')
print(price)

[100, 200, 300]


На второй итерации начали реализовывать аналитику.

Для начала я продемонстрирую плохой пример реализации, который не будет подчиняться `SRP`.

Программист решил реализовать аналитику прямо внутри класса с базой данных.

In [4]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column
    
    def mean_in_column(self, column_name):
        column = self.select_column(column_name)
        # Обратите внимание, я считаю среднее
        # таким образом не просто так.
        # Это нарочный баг.
        # Баги встречаются везде и это нормально.
        # Хоть здесь он и совсем глупый.
        return sum(column) / 3
    
    def max_in_column(self, column_name):
        column = self.select_column(column_name)
        return max(column)

In [5]:
db = DB('0.0.0.0', DATA)

print(db.mean_in_column('price'))
print(db.max_in_column('price'))

200.0
300


Но что же не так с этим классом?

То, что он содержит разную "бизнес" логику:
* коннекст к БД
* Аналитику

Прошло время. Кому то понадобилось считать аналитику не из базы данных, а напрямую из списка:

In [6]:
ANOTHER_PRICES = [101, 404, 228]

Программист видит код аналитики в `DB`, но он реализован так, что сложно или невозможно переиспользовать его на списке `ANOTHER_PRICES`.

Проще всего взять и втупую сделать копипасту:

In [7]:
class Analytics:
    def mean(self, distribution):
        return sum(distribution) / 3
    
    def max(self, distribution):
        return max(distribution)

In [8]:
analytics = Analytics()

print(analytics.mean(ANOTHER_PRICES))
print(analytics.max(ANOTHER_PRICES))

244.33333333333334
404


Вдруг! Тестировщик нашел баг (или не тестировщик, а клиент нашей программы). Оказывается мы считаем неправильно среднее:

In [9]:
ANOTHER_DATA = {
    'price': [1,1,1,1],
    'item': ['пиво', 'рыба', 'раки', 'кальмар']
}

In [10]:
db = DB('0.0.0.0', ANOTHER_DATA)

assert db.mean_in_column('price') == 1

AssertionError: 

Мы пойдем исправлять `DB`.

Затем, увидим, что в `Analytics` тоже есть баг.

Пойдем исправлять `Analytics`.

...

Все это стоит ресурсов и можно было этого избежать.

Вместо расширения функционала изначальной базы данных, стоило создать два отдельных класса с единственной ответственностью, т.е. руководствоваться `SRP`:

In [11]:
class DB:
    def __init__(self, connection_string, data):
        self.__connect = self.__connect_to_db(connection_string)
        self.__data = data
        
    def __connect_to_db(self, connection_string):
        if len(connection_string.split('.')) == 4:
            return connection_string
        else:
            raise ValueError
        
    def select_column(self, name):
        column = self.__data[name]
        return column


class Analytics:
    def mean(self, distribution):
        # Обратите внимание, я считаю среднее
        # таким образом не просто так.
        # Это нарочный баг.
        # Баги встречаются везде и это нормально.
        # Хоть здесь он и совсем глупый.
        return sum(distribution) / 3
    
    def max(self, distribution):
        return max(distribution)


class AnalyticsOnDB:
    def __init__(self, db, analytics):
        self.__db = db
        self.__analytics = analytics
        
    def mean_in_column(self, column_name):
        column = self.__db.select_column(column_name)
        return self.__analytics.mean(column)
    
    def max_in_column(self, column_name):
        column = self.__db.select_column(column_name)
        return self.__analytics.mean(column)

И теперь у нас есть возможность использовать отдельно каждый класс.

In [12]:
db = DB('0.0.0.0', DATA)
print(db.select_column('price'))

analytics = Analytics()

print(analytics.mean(ANOTHER_PRICES))
print(analytics.max(ANOTHER_PRICES))

[100, 200, 300]
244.33333333333334
404


При этом, в той же ситуации с обнаруженным багом:

In [13]:
db = DB('0.0.0.0', ANOTHER_DATA)
analytics = Analytics()

db_analytics = AnalyticsOnDB(db, analytics)
assert db_analytics.mean_in_column('price') == 1

AssertionError: 

Мы исправим ошибку в `Analytics.mean` и все зависимые классы будут прекрасно работать:

In [14]:
class Analytics:
    def mean(self, distribution):
        return sum(distribution) / len(distribution)
    
    def max(self, distribution):
        return max(distribution)

In [15]:
db = DB('0.0.0.0', ANOTHER_DATA)
analytics = Analytics()

db_analytics = AnalyticsOnDB(db, analytics)
assert db_analytics.mean_in_column('price') == 1

_____
Тем не менее, всегда стоит помнить, что `ООП` и `SOLID` не всегда необходимы. Иногда это приводит к сильному оверинженерингу. В частности, через чур руководствуясь этим принципом можно наплодить очень много сущностей, с которыми будет тяжело работать, поддерживать и вообще ориентироваться в проекте.